## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/06/29


#### 这是2024年高校大数据挑战赛的baseline,你可以在<a href="https://github.com/yunsuxiaozi/AI-and-competition">AI and competition</a>里获取更多比赛的baseline。本次比赛官网如下:<a href="https://www.heywhale.com/org/bdc/competition/area/6662bf9a8d6c97c5d0c6bb10/leaderboard">2024bdc</a>

#### 本次比赛官方所给的baseline是最新的论文iTransformer,分数非常高,大部分的选手都已经采用,并且在它的基础上改进取得了更好的成绩。我这里使用2维的CNN来做一个简单的baseline,分数不高,仅供参考。


#### 本次比赛的数据集已经被老师上传到Kaggle了,数据集链接如下:<a href="https://www.kaggle.com/datasets/bruceqdu/bigdata2024">2024bdc dataset</a>。这里也直接在Kaggle上运行程序,如果想使用我的baseline可以将数据的路径改成你自己的路径。

## 1.导入必要的python库,这里不多做解释,注释也已经写的很清楚了。固定随机种子是为了保证模型可以复现。

In [1]:
import numpy as np#矩阵运算与科学计算的库
from sklearn.model_selection import train_test_split
import torch#深度学习库,pytorch
import torch.nn as nn#neural network,神经网络
import torch.nn.functional as F#神经网络函数库
import torch.optim as optim#一个实现了各种优化算法的库
import gc#垃圾回收模块
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

#设置随机种子
import random
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True#将cuda加速的随机数生成器设为确定性模式
    torch.backends.cudnn.benchmark = False#关闭CuDNN框架的自动寻找最优卷积算法的功能，以避免不同的算法对结果产生影响
    torch.manual_seed(seed)#pytorch的随机种子
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

## 2.导入数据。这里有必要对数据进行详细的说明。

- global_data:shape为(5848, 4, 9, 3850)

 5848是时间,数据是2019年1月-2020年12月每3小时一次,2年总共有731天,每天有8次记录,故731\*8=5848
 
 4是4个特征,即:十米高度的矢量纬向风速10U，正方向为东方(m/s);十米高度的矢量经向风速10V，正方向为北方(m/s);两米高度的温度值T2M（℃）;均一海平面气压MSL（Pa）
 
 9是9个网格,即:左上、上、右上、左、中、右、左下、下、右下.
 
 3850就是3850个站点。
 
- temp:shape为(17544, 3850, 1)
 
 17544为5848\*3,就是把数据变成1小时1次记录
 
 3850 是3850个站点
 
 1我感觉这个维度完全多余。
 
- wind:shape为(17544, 3850, 1),解释和temp一样。


#### 对数据的处理:

- 首先需要将global_data在时间上变成1小时记录一次,这里由于就是baseline,所以将1个数据复制3次,如果后续改进,可以尝试用插值来搞? 9这个维度是方位,由于每个方位检测到的4个特征应该都是相关性比较高的,所以这里考虑直接对它们求平均处理。这样就将global_data变成(17544,4,1,3850)。由于后续的处理是根据每个站点前144个小时的特征预测下一个小时的特征,所以,这里将数据变成(3850,17544,4),具体怎么变见代码。

- 对于temp和wind的处理就是将(17544,3850,1)变成(3850,17544,1)


In [2]:
path='/kaggle/input/bigdata2024/global/'
#每3个小时,(温度、湿度、风速、风向),(左上、上、右上、左、中、右、左下、下、右下),3850个站点
# (5848, 4, 9, x)
global_data=np.load(path+"global_data.npy").mean(axis=-2,keepdims=True)
print(f"global_data.shape:{global_data.shape}")
#将3个小时变成1个小时  (5848*3, 4, 1, x)
global_data_hour=np.repeat(global_data, 3, axis=0)
    
del global_data
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存

# (5848*3, 4, 9, x)->(x,5848*3,36)
global_data_hour=global_data_hour.transpose(3,0,1,2)
#(x,5848*3,36)
global_data_hour=global_data_hour.reshape(len(global_data_hour),-1,4)
print(f"global_data_hour.shape:{global_data_hour.shape}")

#每个小时,每个站点的温度 (17544, x, 1)->(x,17544,1)
temp=np.load(path+"temp.npy").transpose(1,0,2)
print(f"temp.shape:{temp.shape}")
#每个小时,每个站点的风速 (17544, x, 1)->(x,17544,1)
wind=np.load(path+"wind.npy").transpose(1,0,2)
print(f"wind.shape:{wind.shape}")

global_data.shape:(5848, 4, 1, 3850)
global_data_hour.shape:(3850, 17544, 4)
temp.shape:(3850, 17544, 1)
wind.shape:(3850, 17544, 1)


## 3.数据的采样。

#### 我们之前得到的特征是global_data:(3850,17544,4),temp和wind(3850,17544,1),我们这里的idea是用前144个时刻的所有特征预测下一个时刻的temp和wind,所以先拼接一个总特征(3850,17544,6),然后再构造X和y1,y2。由于全部数据的数据量巨大,所以这里对数据进行采样,采样的概率为0.0125,因为0.015我试过,超内存了。对数据进行标准化是神经网络必要的数据预处理,train_mean和train_std也要保存,因为提交的时候对测试数据也要进行同样的操作。最后数据处理完的维度X:(len(X),144\*6),y1,y2:(len(X),1)

In [3]:
#(x,17544,38)
train_feats=np.concatenate((global_data_hour,temp,wind),axis=-1)
print(f"train_feats.shape:{train_feats.shape}")
#(x,17544,1),(x,17544,1)
label1,label2=temp,wind

def get_train_data(train_feats,label1,label2):#(x,17544,38),(x,17544,1),(x,17544,1)
    X,y1,y2=[],[],[]
    #每个站点
    for si in range(train_feats.shape[0]):
        for ti in range(train_feats.shape[1]-144):
            if np.random.rand()<0.0125:#这里再进行采样
                #si个站点ti:ti+144个时刻的所有特征
                X.append(train_feats[si][ti:ti+144].reshape(-1))
                y1.append(label1[si][ti+144])
                y2.append(label2[si][ti+144])
    X,y1,y2=np.array(X),np.array(y1),np.array(y2)
    return X,y1,y2
X,y1,y2=get_train_data(train_feats,label1,label2)
train_mean=X.mean(axis=0)
train_std=X.std(axis=0)
np.save("train_mean.npy",train_mean)
np.save("train_std.npy",train_std)
X=(X-train_mean)/train_std
print(f"X.shape:{X.shape},y1.shape:{y1.shape},y2.shape:{y2.shape}")
del global_data_hour,temp,wind,train_feats
gc.collect()#手动触发垃圾回收,强制回收由垃圾回收器标记为未使用的内存

train_feats.shape:(3850, 17544, 6)
X.shape:(836534, 864),y1.shape:(836534, 1),y2.shape:(836534, 1)


0

## 4.BaselineModel

#### 这里搭建了一个简单的CNN作为baseline。

In [4]:
class BaselineModel(nn.Module):
    def __init__(self,):
        super(BaselineModel,self).__init__()
        self.conv=nn.Sequential(
                  #1*24*36->16*24*36
                  nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1,padding=1),
                  nn.BatchNorm2d(16),
                  #16*24*36->16*12*18
                  nn.MaxPool2d(kernel_size=2,stride=2),
                  nn.GELU(),
                  #16*12*18->32*12*18
                  nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1),
                  nn.BatchNorm2d(32),
                  #32*12*18->64*12*18
                  nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,stride=1,padding=2),
                  nn.BatchNorm2d(64),
                  #64*12*18->64*6*9
                  nn.MaxPool2d(kernel_size=2,stride=2),
                  nn.GELU(),
                  #64*6*9->128*6*9
                  nn.Conv2d(in_channels=64,out_channels=128,kernel_size=5,stride=1,padding=2),
                  nn.BatchNorm2d(128),
                  #128*6*9->128*3*4
                  nn.MaxPool2d(kernel_size=2,stride=2),
                  nn.GELU(),
        )
        self.head=nn.Sequential(
                nn.Linear(128*3*4,128),
                nn.BatchNorm1d(128),
                nn.GELU(),
                nn.Linear(128,256),
                nn.BatchNorm1d(256),
                nn.GELU(),
                nn.Linear(256,1)
        )
        
    def forward(self,x):
        x=self.conv(x)
        x=x.reshape(x.shape[0],-1)
        return self.head(x)

## 5.模型的训练

#### date就是说这是6月29日的第二次提交。这里之所以将864reshape成(1,24,36)只是想将数据搞得尽可能正方形一点,好使用CNN来卷积,模型训练使用的是MSE,评估指标使用的是官方的评估指标,由于是对temp和wind搞了2个模型,所以没有看最终指标的分数。可能是因为我用train_test_split存在数据泄露的情况,线下跑出来的指标好低,和线上完全对不上。

In [5]:
date='0629_2'
def loss_fn(y_true,y_pred):#torch.tensor
    return torch.mean((y_true-y_pred)**2)
def metric(y_true,y_pred):#np.array
    return np.mean((y_true-y_pred)**2)/np.var(y_true)

def train(X,y,batch_size=1024,num_epochs=5,name='wind'):#传入的是np.array的数据,name是wind还是temp
    train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state=2024,shuffle=False)
    #模型设置
    model=BaselineModel()
    #优化器设置
    optimizer=optim.Adam(model.parameters(),lr=0.000025,betas=(0.5,0.999))
    for epoch in range(num_epochs):
        print(f"epoch:{epoch},name:{name}")
        #模型设置为训练状态
        model.train()
        #将梯度清空
        optimizer.zero_grad()
        #每次训练之前先打乱顺序
        random_index=np.arange(len(train_X))
        np.random.shuffle(random_index)
        train_X,train_y=train_X[random_index],train_y[random_index]
        train_loss=0.0
        for idx in range(0,len(train_X),batch_size):
            train_X1=torch.Tensor(train_X[idx:idx+batch_size]).reshape(-1,1,24,36)
            train_y1=torch.Tensor(train_y[idx:idx+batch_size])
            train_pred=model(train_X1)
            loss=loss_fn(train_y1,train_pred)
            #反向传播
            loss.backward()
            #优化器进行优化(梯度下降,降低误差)
            optimizer.step()
            train_loss+=loss
        print(f"train_loss:{train_loss/(len(train_X)//batch_size)}")
         #模型设置为评估模式
        model.eval()
        with torch.no_grad():
            valid_loss=0.00
            valid_preds=np.zeros(len(valid_y))
            for idx in range(0,len(valid_X),batch_size):
                valid_X1=torch.Tensor(valid_X[idx:idx+batch_size]).reshape(-1,1,24,36)
                valid_y1=torch.Tensor(valid_y[idx:idx+batch_size])
                valid_pred=model(valid_X1)
                loss=loss_fn(valid_y1,valid_pred)
                valid_loss+=loss
                valid_preds[idx:idx+batch_size]=valid_pred.detach().numpy().reshape(-1)
            print(f"valid_loss:{valid_loss/(len(valid_X)//batch_size)},metric:{metric(valid_y.reshape(-1),valid_preds)}")
        torch.cuda.empty_cache()
    torch.save(model.state_dict(),f"{date}{name}.pth")
train(X,y1,batch_size=128,num_epochs=10,name='temp')
train(X,y2,batch_size=128,num_epochs=10,name='wind')

epoch:0,name:temp
train_loss:39.63254928588867
valid_loss:6.957380294799805,metric:0.042841896378927866
epoch:1,name:temp
train_loss:6.404951572418213
valid_loss:4.364864349365234,metric:0.026876498608155396
epoch:2,name:temp
train_loss:3.9735267162323
valid_loss:3.3389687538146973,metric:0.020568538182798583
epoch:3,name:temp
train_loss:3.416158437728882
valid_loss:3.1424412727355957,metric:0.019356603391767334
epoch:4,name:temp
train_loss:3.0788538455963135
valid_loss:2.7104780673980713,metric:0.01669682685350834
epoch:5,name:temp
train_loss:2.764535903930664
valid_loss:3.2275211811065674,metric:0.0198797962722501
epoch:6,name:temp
train_loss:2.61907696723938
valid_loss:3.0715019702911377,metric:0.018926934101076896
epoch:7,name:temp
train_loss:2.4271206855773926
valid_loss:2.6960549354553223,metric:0.016605464776724428
epoch:8,name:temp
train_loss:2.1649467945098877
valid_loss:2.8013415336608887,metric:0.017257515111020413
epoch:9,name:temp
train_loss:2.076241970062256
valid_loss:2.

## 6.模型的预测


#### 这里就是线下按照测试数据的大小随机生成测试数据来跑一下,看看能不能跑通模型,这段代码也可以写入提交的index.py文件里。

#### 由于我对代码也一直在改动,注释里的内容也不一定是现在的版本的注释,各位看懂就好,不要在意注释中的错误。


In [6]:
#测试数据
x=60#60个观测站点
#71个不连续的周,56(每3个小时测一次),观测4个特征,9个观测方位,x个站点
cenn_data=np.random.randn(71,56,4,9,x).mean(axis=-2,keepdims=True)#真实情况是np.load加载的
print(f"cenn_data.shape:{cenn_data.shape}")
#将3个小时变成1个小时  (71,168,4,1,x)
cenn_data_hour=np.repeat(cenn_data, 3, axis=1)
cenn_data_hour=cenn_data_hour.transpose(0,4,1,2,3)#71*x*168*4*9
cenn_data_hour=cenn_data_hour.reshape(71*x,168,4)


#cenn/temp_lookback.npy 71个不连续的周  1个小时一次  x站上一周的温度
temp_lookback=np.random.randn(71,168,x,1)
print(f"temp_lookback.shape:{temp_lookback.shape}")
temp_lookback=temp_lookback.transpose(0,2,1,3)#71,x,168,1
temp_lookback=temp_lookback.reshape(71*x,168,1)
#cenn/wind_lookback.npy 71个不连续的周  1个小时一次  x站上一周的风速
wind_lookback=np.random.randn(71,168,x,1)
print(f"wind_lookback.shape:{wind_lookback.shape}")
wind_lookback=wind_lookback.transpose(0,2,1,3)#71,x,168,1
wind_lookback=wind_lookback.reshape(71*x,168,1)

#71*x个站点,168小时,38个特征
total_feats=np.concatenate((cenn_data_hour,temp_lookback,wind_lookback),axis=-1)

def predict(feats,model_name='temp.pth',batch_size=128):
    model = BaselineModel()
    model.load_state_dict(torch.load(model_name))
    #预测24个小时的用的是71*x个站点(i:i-24)时刻的36个特征
    data=[]
    for i in range(24):
        data.append(feats[:,i:i-24,].reshape(feats.shape[0],-1))
    data=np.array(data)#24个小时,71*x个站点 【i:i-24时刻*36个特征】
    #【24小时*71*x个站点】*【i:i-24时刻*36个特征】
    data=data.reshape(-1,data.shape[-1])
    print(f"input.shape:{data.shape}")
    data=(data-train_mean)/train_std
    #test_preds=【24小时*71*x个站点】*【预测值】
    test_preds=np.zeros(len(data))
    for idx in range(0,len(data),batch_size):
        data1=torch.Tensor(data[idx:idx+batch_size]).reshape(-1,1,24,36)
        test_preds[idx:idx+batch_size]=model(data1).detach().numpy().reshape(-1)
    test_preds=test_preds.reshape(24,71,x,-1)
    #71个周,24小时,x个站点的预测值
    test_preds=test_preds.transpose(1,0,2,3)
    return test_preds
test_preds1=predict(feats=total_feats,model_name=f'{date}temp.pth',batch_size=128)
test_preds2=predict(feats=total_feats,model_name=f'{date}wind.pth',batch_size=128)
test_preds1.shape,test_preds2.shape

cenn_data.shape:(71, 56, 4, 1, 60)
temp_lookback.shape:(71, 168, 60, 1)
wind_lookback.shape:(71, 168, 60, 1)
input.shape:(102240, 864)
input.shape:(102240, 864)


((71, 24, 60, 1), (71, 24, 60, 1))